# Extract WMAP instrument parameters

[Executed version of this notebook on Gist](https://gist.github.com/zonca/7b00e6ae5d14bdfec33cc992cd7d4554)

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from Lambda WMAP DR5",
]

In [ ]:
import numpy as np
from astropy.io import fits

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from pathlib import Path

# Check if the file does not exist before downloading
file_path = Path("wmap_ampl_bl_9yr_v5p1.tar.gz")
if not file_path.exists():
    !wget https://lambda.gsfc.nasa.gov/data/map/dr5/ancillary/beams/wmap_ampl_bl_9yr_v5p1.tar.gz
    !tar xzvf wmap_ampl_bl_9yr_v5p1.tar.gz

In [ ]:
%ls *V1*

In [ ]:
freq_bands = ["K", "Ka", "Q", "V", "W"]

In [ ]:
from pathlib import Path

# Get all files matching the pattern
files = Path(".").glob("wmap_ampl_bl*txt")

# Extract channel names from filenames
channels = [file.stem.split("_")[3] for file in files]

print(channels)

## Create the beam files

In [ ]:
ls *V1*

In [ ]:
!head -n 10 wmap_ampl_bl_V1_9yr_v5p1.txt

In [ ]:
channels_by_freq = {}
for freq_band in freq_bands:
    channels_by_freq[freq_band] = []
    for channel in channels:
        if freq_band == "K" and channel.startswith("Ka"):
            continue
        if freq_band in channel:
            channels_by_freq[freq_band].append(channel)
channels_by_freq

In [ ]:
from astropy.table import QTable

beam = {}
for freq_band, band_channels in channels_by_freq.items():
    combined_data = None
    for ch in band_channels:
        wmap_data = np.loadtxt(f"wmap_ampl_bl_{ch}_9yr_v5p1.txt")
        if combined_data is None:
            combined_data = wmap_data
        else:
            combined_data[:, 1] += wmap_data[:, 1]
    combined_data[:, 1] /= len(band_channels)
    beam[freq_band] = QTable(
        data=[combined_data[:, 0], combined_data[:, 1]], names=["ell", "B"]
    )
    beam[freq_band]["B"] /= np.max(beam[freq_band]["B"])
    beam[freq_band].meta["comments"] = ipac_warning
    beam[freq_band].write(f"beam_{freq_band}.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
# Given FWHM values in degrees for each frequency band
fwhm_deg_values = {"K": 0.88, "Ka": 0.66, "Q": 0.51, "V": 0.35, "W": 0.22}

# From https://lambda.gsfc.nasa.gov/product/wmap/dr5/

# Convert FWHM from degrees to arcminutes
fwhm_arcmin_values = {band: value * 60 for band, value in fwhm_deg_values.items()}

print(fwhm_arcmin_values)

In [ ]:
for freq_band in freq_bands:
    plt.figure()
    plt.title(f"Beam for {freq_band} band (Log-Log)")
    for ch in channels_by_freq[freq_band]:
        beam_data = np.loadtxt(f"wmap_ampl_bl_{ch}_9yr_v5p1.txt")
        plt.loglog(beam_data[:, 0], beam_data[:, 1], label=f"{ch} (single)")
    plt.loglog(
        beam[freq_band]["ell"],
        beam[freq_band]["B"],
        label=f"{freq_band} (averaged)",
        linewidth=2,
        color="black",
    )
    plt.legend()
    plt.grid()
    plt.xlabel("Multipole moment (ell)")
    plt.ylabel("Beam response (B)")
    plt.show()

## Create bandpass files

As suggested in https://lambda.gsfc.nasa.gov/product/wmap/dr5/bandpass_info.html, we combine the bandpasses after normalizing them to unit integral, we create a single bandpass file for each frequency band

In [ ]:
from pathlib import Path

# Define the file path
file_path = Path("wmap_bandpass_v5.tar.gz")

# Check if the file does not exist before downloading
if not file_path.exists():
    !wget https://lambda.gsfc.nasa.gov/data/map/dr5/ancillary/response/wmap_bandpass_v5.tar.gz
    !tar xzvf wmap_bandpass_v5.tar.gz

In [ ]:
# List all files matching the pattern
bandpass_files = list(Path(".").glob("wmap_bandpass_*.cbp"))

# Extract the V21 part from the filenames
bandpass_channels = [file.stem.split("_")[2] for file in bandpass_files]

print(bandpass_channels)

In [ ]:
!head -n 20 wmap_bandpass_V22_v5.cbp

In [ ]:
# Load the data from the file
bandpass_data = np.loadtxt("wmap_bandpass_V21_v5.cbp")

# Display the first few rows of the data
print(bandpass_data[:5])

In [ ]:
from pysm3 import units as u

In [ ]:
from astropy.table import QTable

bandpass = {}
for freq_band in freq_bands:
    combined_data = None
    for ch in bandpass_channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            bandpass_data = np.loadtxt(f"wmap_bandpass_{ch}_v5.cbp")
            # normalize to integral of 1
            bandpass_data[:, 1] /= np.trapz(bandpass_data[:, 1], bandpass_data[:, 0])
            bandpass_data[:, 2] /= np.trapz(bandpass_data[:, 2], bandpass_data[:, 0])
            if combined_data is None:
                combined_data = bandpass_data[:, 1] + bandpass_data[:, 2]
            else:
                combined_data += bandpass_data[:, 1] + bandpass_data[:, 2]
    combined_data /= (
        bandpass_data[:, 0] ** 2
    )  # go from bandpass in RJ to bandpass in power
    combined_data /= np.trapz(combined_data, bandpass_data[:, 0])
    bandpass[freq_band] = QTable(
        names="bandpass_frequency bandpass_weight".split(),
        units=[u.GHz, None],
        data=[bandpass_data[:, 0], combined_data],
    )
    bandpass[freq_band].meta["comments"] = ipac_warning
    bandpass[freq_band].write(
        f"bandpass_{freq_band}.tbl", format="ascii.ipac", overwrite=True
    )

In [ ]:
effective_frequency = {}

for freq_band in freq_bands:
    bandpass_data = bandpass[freq_band]
    frequency = bandpass_data["bandpass_frequency"]
    weight = bandpass_data["bandpass_weight"]

    # Calculate the effective frequency
    eff_freq = np.trapz(frequency * weight, frequency) / np.trapz(weight, frequency)
    effective_frequency[freq_band] = eff_freq

print(effective_frequency)

In [ ]:
for freq_band in freq_bands:
    plt.figure()
    plt.title(f"Bandpass for {freq_band} band (Linear)")
    for ch in bandpass_channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            bandpass_data = np.loadtxt(f"wmap_bandpass_{ch}_v5.cbp")

            # plot bandpass_data[:,1] and 2 against 0
            bandpass_data[:, 1] /= np.trapz(bandpass_data[:, 1], bandpass_data[:, 0])
            bandpass_data[:, 2] /= np.trapz(bandpass_data[:, 2], bandpass_data[:, 0])
            plt.plot(bandpass_data[:, 0], bandpass_data[:, 1], label=f"{ch} 1")
            plt.plot(bandpass_data[:, 0], bandpass_data[:, 2], label=f"{ch} 2")

    # Overplot the averaged bandpass
    bandpass_rj = (
        bandpass[freq_band]["bandpass_weight"]
        * bandpass[freq_band]["bandpass_frequency"] ** 2
    )
    bandpass_rj /= np.trapz(bandpass_rj, bandpass[freq_band]["bandpass_frequency"])
    plt.plot(
        bandpass[freq_band]["bandpass_frequency"],
        bandpass_rj,
        label=f"{freq_band} (averaged)",
        linewidth=2,
        color="black",
    )
    plt.legend()
    plt.grid()
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Bandpass")
    plt.show()

    plt.figure()
    plt.title(f"Bandpass for {freq_band} band (Semilog)")
    for ch in bandpass_channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            bandpass_data = np.loadtxt(f"wmap_bandpass_{ch}_v5.cbp")
            # normalize to unit integral
            bandpass_data[:, 1] /= np.trapz(bandpass_data[:, 1], bandpass_data[:, 0])
            bandpass_data[:, 2] /= np.trapz(bandpass_data[:, 2], bandpass_data[:, 0])

            # plot bandpass_data[:,1] and 2 against 0
            plt.semilogy(bandpass_data[:, 0], bandpass_data[:, 1], label=f"{ch} 1")
            plt.semilogy(bandpass_data[:, 0], bandpass_data[:, 2], label=f"{ch} 2")

    # Overplot the averaged bandpass
    plt.semilogy(
        bandpass[freq_band]["bandpass_frequency"],
        bandpass[freq_band]["bandpass_weight"],
        label=f"{freq_band} (averaged)",
        linewidth=2,
        color="black",
    )
    plt.legend()
    plt.grid()
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Bandpass")
    plt.show()

## Create the instrument model

In [ ]:
table = QTable(
    names=[
        "telescope",
        "band",
        "fwhm",
        "center_frequency",
        "nside",
        "bandpass_file",
        "beam_file",
    ],
    dtype=[str, str, float, float, int, str, str],
    units=[None, None, u.arcmin, u.GHz, None, None, None],
)

In [ ]:
from collections import OrderedDict

for label in freq_bands:
    table.add_row(
        OrderedDict(
            telescope="WMAP",
            band=label,
            nside=512,
            fwhm=fwhm_arcmin_values[label] * u.arcmin,
            center_frequency=effective_frequency[label],
            bandpass_file="bandpass_" + label + ".tbl",
            beam_file="beam_" + label + ".tbl",
        )
    )

In [ ]:
table

In [ ]:
table.meta["comments"] = ipac_warning
table.write(f"instrument_model.tbl", format="ascii.ipac", overwrite=True)